# Lesson 3: Reflection and Blogpost Writing

## Setup

In [ ]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [ ]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [ ]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [ ]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [ ]:
print(reply)

Title: Unlocking the Power of DeepLearning.AI

Dive into the world of Artificial Intelligence with DeepLearning.AI! Offering cutting-edge courses created by industry experts, this platform equips you with the skills to thrive in the age of AI. From computer vision to natural language processing, DeepLearning.AI covers it all. Whether you're a beginner or a seasoned professional, there's something for everyone. Join a community of learners worldwide and stay ahead of the curve. Embrace the future with DeepLearning.AI and unleash your potential in this rapidly evolving field. Start your journey today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [ ]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [ ]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unlocking the Power of DeepLearning.AI

Dive into the world of Artificial Intelligence with DeepLearning.AI! Offering cutting-edge courses created by industry experts, this platform equips you with the skills to thrive in the age of AI. From computer vision to natural language processing, DeepLearning.AI covers it all. Whether you're a beginner or a seasoned professional, there's something for everyone. Join a community of learners worldwide and stay ahead of the curve. Embrace the future with DeepLearning.AI and unleash your potential in this rapidly evolving field. Start your journey today!

--------------------------------------------------------------------------------
Critic (to Writer):

The blogpost effectively intro

## Nested chat

In [ ]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [ ]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [ ]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [ ]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [ ]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [ ]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [ ]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unlocking the Power of DeepLearning.AI

Dive into the world of Artificial Intelligence with DeepLearning.AI! Offering cutting-edge courses created by industry experts, this platform equips you with the skills to thrive in the age of AI. From computer vision to natural language processing, DeepLearning.AI covers it all. Whether you're a beginner or a seasoned professional, there's something for everyone. Join a community of learners worldwide and stay ahead of the curve. Embrace the future with DeepLearning.AI and unleash your potential in this rapidly evolving field. Start your journey today!

--------------------------------------------------------------------------------

**************************************************


--------------------------------------------------------------------------------
Critic (to Writer):

Aggregated feedback from reviewers:
1. SEO Specialists:
   - Content is informative and engaging, highlighting key features and benefits of DeepLearning.AI.
   - Suggested improvements: incorporate targeted keywords, add meta tags, include internal links for SEO optimization.

2. Ethics Reviewer:
   - Ensure content accurately represents the breadth of courses without exaggeration.
   - Disclose any affiliations or sponsorships with DeepLearning.AI for transparency.
   - Avoid making promises of guaranteed success that may create unrealistic expectations.

Final suggestions:
- The content is praised for being informative and engaging, focusing on the benefits of DeepLearning.AI.
- Incorporate targeted keywords and meta tags for better SEO optimization.
- Ensure transparency by disclosing any affiliations or sponsorships with DeepLearning.AI.
- Avoid making unrealistic promises of guar

## Get the summary

In [ ]:
print(res.summary)

Title: Unveiling the Potential of DeepLearning.AI for Aspiring Minds

Embark on a transformative journey with DeepLearning.AI, delving into the realm of Artificial Intelligence through expert-led courses. From computer vision to natural language processing, master the skills essential in today's AI-driven world. Engage with a global community of learners, embracing continuous growth and innovation. Transparency is key - we disclose any affiliations, ensuring trust and credibility. While we don't promise overnight success, we offer invaluable knowledge and resources for your professional advancement. Optimize your learning experience by incorporating targeted keywords, meta tags, and internal links. Join us today and discover the limitless possibilities of DeepLearning.AI.
